In [3]:
import os, shutil

import keras
from keras.constraints import max_norm

import matplotlib.pyplot as plt
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [13]:
original_data_dir = './dog-breeds'
base_dir = './load-data'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

In [14]:
os.mkdir(base_dir)
os.mkdir(train_dir)
os.mkdir(validation_dir)
os.mkdir(test_dir)

In [15]:
dog_breed_dir_list = os.listdir(original_data_dir)
dog_breeds = {}

In [16]:
for dog_breed_dir in dog_breed_dir_list:
    DIR = os.path.join(original_data_dir, dog_breed_dir)

    size = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
    dog_breeds[dog_breed_dir] = {'size': size}

In [17]:
for dog_breed in dog_breeds:
    TEN_PERCENT = round(dog_breeds[dog_breed]['size'] * 0.1)
    EIGHTY_PERCENT = round(dog_breeds[dog_breed]['size'] * 0.8)

    validation_size = TEN_PERCENT
    test_size = TEN_PERCENT + TEN_PERCENT
    training_size = TEN_PERCENT + TEN_PERCENT + EIGHTY_PERCENT

    dog_breed_validation_dir = os.path.join(validation_dir, dog_breed)
    dog_breed_test_dir = os.path.join(test_dir, dog_breed)
    dog_breed_train_dir = os.path.join(train_dir, dog_breed)

    os.mkdir(dog_breed_validation_dir)
    os.mkdir(dog_breed_test_dir)
    os.mkdir(dog_breed_train_dir)

    for x in ['{}.jpg'.format(i) for i in range(1, validation_size)]:
        src = os.path.join(original_data_dir, dog_breed, x)
        dst = os.path.join(dog_breed_validation_dir, x)
        shutil.copyfile(src, dst)

    for x in ['{}.jpg'.format(i) for i in range(validation_size, test_size)]:
        src = os.path.join(original_data_dir, dog_breed, x)
        dst = os.path.join(dog_breed_test_dir, x)
        shutil.copyfile(src, dst)

    for x in ['{}.jpg'.format(i) for i in range(test_size, training_size)]:
        src = os.path.join(original_data_dir, dog_breed, x)
        dst = os.path.join(dog_breed_train_dir, x)
        shutil.copyfile(src, dst)

In [18]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu', kernel_constraint=max_norm(3),
                        input_shape=(150, 150, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu', kernel_constraint=max_norm(3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu', kernel_constraint=max_norm(3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu', kernel_constraint=max_norm(3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(120, activation='softmax')
])

/Users/jeromedelosreyes/Desktop/pythonProject/.venv/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [19]:
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 150, 150, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 75, 75, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 75, 75, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 37, 37, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 37, 37, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 18, 18, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 18, 18, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 9, 9, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 20736)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │    10,617,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 120)            │        15,480 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,185,464 (42.67 MB)

 Trainable params: 11,185,464 (42.67 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=256,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=256,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=256,
    class_mode='categorical'
)

Found 16493 images belonging to 120 classes.
Found 1944 images belonging to 120 classes.
Found 2064 images belonging to 120 classes.


In [22]:
history = model.fit(
    train_generator,
    steps_per_epoch=256,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=256,
    verbose=1
)

Epoch 1/50


/Users/jeromedelosreyes/Desktop/pythonProject/.venv/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-04-11 00:33:38.769093: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 5 of 8
2024-04-11 00:33:45.734058: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


 65/256 ━━━━━━━━━━━━━━━━━━━━ 30:31 10s/step - accuracy: 0.0091 - loss: 4.7961

2024-04-11 00:44:08.952952: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/contextlib.py:135: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(type, value, traceback)


256/256 ━━━━━━━━━━━━━━━━━━━━ 666s 2s/step - accuracy: 0.0094 - loss: 4.7913 - val_accuracy: 0.0103 - val_loss: 4.7850
Epoch 2/50


2024-04-11 00:44:29.649277: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-04-11 00:44:41.560354: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:20: Filling up shuffle buffer (this may take a while): 4 of 8
2024-04-11 00:44:53.909184: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


 22/256 ━━━━━━━━━━━━━━━━━━━━ 39:57 10s/step - accuracy: 0.0138 - loss: 4.7852

KeyboardInterrupt: 

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)

In [ ]:
test_accuracy

In [ ]:
# Define a function to plot the results
def result_plotting():
    plt.figure(figsize=[8, 6])
    plt.plot(history.history['accuracy'], 'blue', linewidth=3.0)
    plt.plot(history.history['val_accuracy'], 'red', ls='--', linewidth=3.0)
    plt.legend(['Training Accuracy', 'Validation Accuracy'], fontsize=18, loc='lower right')
    plt.xlabel('Epochs', fontsize=16)
    plt.ylabel('Accuracy', fontsize=16)
    plt.title('Accuracy Curves', fontsize=16)

In [ ]:
result_plotting()